In [1]:
%pip install crewai==0.28.8 crewai_tools==0.1.6 

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install langchain langchain-groq langchain-community

INFO: pip is looking at multiple versions of langchain-groq to determine which version is compatible with other requirements. This could take a while.
  Attempting uninstall: langchain-groq
    Found existing installation: langchain-groq 0.2.2
    Uninstalling langchain-groq-0.2.2:
      Successfully uninstalled langchain-groq-0.2.2
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [35]:
from dotenv import load_dotenv
import os

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")


In [36]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="mixtral-8x7b-32768",
    api_key="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",
    temperature=0.1,
    max_tokens=1000,
)

In [37]:
%pip install groq


Note: you may need to restart the kernel to use updated packages.


In [38]:
from crewai import Agent, Task, Crew

In [39]:
research_agent = Agent(
    role="Company Research Expert",
    goal="Conduct comprehensive research to understand the company’s industry, strategic focus, and offerings.",
    backstory=(
        "You work for aiplanet, providing detailed and professional analysis of companies. "
        "Your job is to understand the company's market position, vision, and growth strategy. "
        "You research the company’s offerings, focusing on strategic areas such as operations, supply chain, and customer experience. "
        "Your work helps generate relevant AI/ML opportunities and understand how AI technologies can support the company's goals."
    ),
    llm=llm,
    verbose=True
)


In [40]:
from langchain_groq import ChatGroq

In [41]:
# Create the Use Case Generation Agent
use_case_generation_agent = Agent(
    role="AI Use Case Analyst",
    goal="Analyze industry trends and propose AI/ML and GenAI use cases for companies.",
    backstory=(
        "You are responsible for identifying relevant AI, ML, and GenAI opportunities based on market research. "
        "You analyze industry trends and propose actionable use cases where AI technologies can enhance processes, "
        "improve customer satisfaction, and increase operational efficiency. "
        "Your goal is to provide value-driven insights tailored to the company's sector and strategic objectives."
    ),
    verbose=True,
    llm=llm  # Pass the Groq LLM
)


In [42]:
# Create the Resource Asset Collection Agent
resource_asset_collection_agent = Agent(
    role="Resource Asset Collector",
    goal="Identify and collect relevant datasets and propose GenAI solutions for the use cases.",
    backstory=(
        "You are responsible for collecting data and tools to implement AI use cases. "
        "Search for relevant datasets on platforms like Kaggle, HuggingFace, and GitHub. "
        "Also, propose additional GenAI solutions such as document search, automated report generation, "
        "and AI-powered chat systems that align with the use cases."
    ),
    verbose=True,
    llm=llm  # Pass the Groq LLM
)

## Tools

In [43]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

In [44]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://www.tesla.com/"
)

In [45]:
industry_research_task = Task(
    description=(
        "{company_name} is a key player in the {industry}. Your task is to gather "
        "information about the company’s key offerings, strategic focus areas, and vision for the future. "
        "You should identify the company’s position in the market, its growth strategy, and its main focus areas, "
        "such as supply chain, operations, or customer experience. Make sure to provide a comprehensive analysis."
    ),
    expected_output=(
        "A detailed, informative report on the company’s strategic focus areas, "
        "including its position in the market, growth strategy, and key offerings. "
        "The report should be comprehensive, citing all sources used, and should provide actionable insights."
    ),
    tools=[docs_scrape_tool],
    agent=research_agent
)


In [46]:
use_case_generation_task = Task(
    description=(
        "Using the research data collected about {company_name} and the {industry}, your task is to "
        "analyze current industry trends and identify where AI/ML and GenAI can bring value. "
        "Propose actionable use cases to enhance customer satisfaction, improve operational efficiency, "
        "and streamline internal processes. Your use cases should focus on AI technologies like machine learning, GenAI, and NLP."
    ),
    expected_output=(
        "A list of AI/ML and GenAI use cases that are directly relevant to {company_name}'s industry, "
        "focusing on enhancing operations, customer experiences, and improving key areas such as supply chain management. "
        "For each use case, ensure to include its potential impact and any necessary tools/technologies needed for implementation."
    ),
    agent=use_case_generation_agent,
)


In [47]:
resource_collection_task = Task(
    description=(
        "For the AI/ML use cases proposed for {company_name} in the {industry}, your task is to identify and "
        "collect relevant datasets and tools from Kaggle, HuggingFace, and GitHub. "
        "Additionally, propose GenAI solutions like automated document search, report generation, and AI-powered chat systems. "
        "Ensure that all resources and solutions align with the company’s needs and operational focus."
    ),
    expected_output=(
        "A list of relevant datasets, tools, and GenAI solutions that can be used to implement the proposed use cases. "
        "Each resource should include a link and a brief description of how it supports the specific use case. "
        "Propose solutions that enhance internal processes, customer support, or any other key areas identified."
    ),
    tools=[docs_scrape_tool],
    agent=resource_asset_collection_agent,
    llm=llm
)


In [48]:
crew= Crew(
    agents=[research_agent,use_case_generation_agent,resource_asset_collection_agent],
    tasks=[industry_research_task,use_case_generation_task,resource_collection_task],
    memory=True
)

2024-12-29 15:07:02,543 - 7444 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [50]:
inputs = {
    "company_name": "Tesla",
    "industry": "Automotive",
    "company_focus": "Supply Chain, Customer Experience, Manufacturing Operations"
}
result = crew.kickoff(inputs=inputs)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: fake. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}